# RAG

Para este ejemplo se mejorará el ejemplo que se creó antes ([aquí](../01-rag/rag.ipynb)), se utilizará la misma fuente de información, pero ahora siguiendo el correcto paso a paso de cómo se debería hacer en realidad (ver [aquí](../rag-explanation.md)). Para este caso, utilizaremos la libreria `langchain` con Claude como LLM, y bases de datos de vectores para almacenar la información de los documentos

## Importar módulos y todo lo necesario

Para el modelo de embedding se puede cambiar a otro en caso de ser necesario

In [2]:
# Tipado
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from typing import TypedDict

# LLM
from langchain_anthropic import ChatAnthropic

# Para el indexing
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_nomic import NomicEmbeddings
from langchain_chroma import Chroma

# Para la generación
from langchain import hub

# Variables de entorno
from os import getenv
from dotenv import load_dotenv

load_dotenv()

CLAUDE_API_KEY = getenv("CLAUDE_API_KEY")
MODEL_NAME = getenv("CLAUDE_MODEL_NAME")
NOMIC_API_KEY = getenv("NOMIC_API_KEY")
LANGCHAIN_API_KEY = getenv("LANGCHAIN_API_KEY")

CHUNK_SIZE = int(getenv("CHUNK_SIZE"))
CHUNK_OVERLAP = int(getenv("CHUNK_OVERLAP"))

In [3]:
llm: ChatAnthropic = ChatAnthropic(
    api_key=CLAUDE_API_KEY,
    model=MODEL_NAME,
    temperature=0,
    max_tokens=2000,
)

text_splitter: RecursiveCharacterTextSplitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP
)

embeddings: NomicEmbeddings = NomicEmbeddings(
    model="gte-multilingual-base",
    nomic_api_key=NOMIC_API_KEY
)

vector_store: Chroma = Chroma(embedding_function=embeddings)

prompt: ChatPromptTemplate = hub.pull("rlm/rag-prompt", api_key=LANGCHAIN_API_KEY)

## Indexación

### Document Loader

In [4]:
loader: TextLoader = TextLoader("./info.md", "utf-8")
docs = loader.load()

### Text Splitter

In [5]:
all_splits: list[Document] = text_splitter.split_documents(docs)
all_splits[:3]

[Document(metadata={'source': './info.md'}, page_content='# NEBULA\n\n## Roadmap\n\n### Purpose and Scope of the SAD\n\nEl SAD tiene como objetivo documentar la arquitectura de Nebula, detallando los componentes estructurales y sus interacciones para cumplir con los requisitos de desarrollo, escalabilidad y seguridad. En este documento se incluye toda la informaci�n sobre la arquitectura de software de Nebula, destacando aquellos aspectos que influyen en el rendimiento y la adaptabilidad del sistema. Las decisiones de dise�o documentadas en el SAD son aquellas que tienen un impacto arquitect�nico relevante en la estructura general del sistema, mientras que los detalles de implementaci�n espec�ficos se documentan en otros informes t�cnicos.\n\n### How the SAD Is Organized'),
 Document(metadata={'source': './info.md'}, page_content='El SAD de Nebula est� estructurado en las siguientes secciones:  \nSección 1: Documentation Roadmap - Proporciona una visi�n general del SAD, el prop�sito de

### Vector Store

In [6]:
store = vector_store.add_documents(all_splits)

In [7]:
store[:3]

['7e584e3d-3f03-4208-b408-eda367e93468',
 'f8ca0ab3-914f-4d2e-a3b1-717418ff3fb6',
 'e13a2de3-7f6f-4ac5-bcc7-c9e31b1350a4']

## Recuperación y Generación

Con `"rlm/rag-prompt"`, se está haciendo este prompt al modelo:

```
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
```

Ver [esto](https://smith.langchain.com/hub/rlm/rag-prompt) para más info


En este caso, para la generación de la respuesta, no se utilizará LangGraph, pero también es posible

In [8]:
class State(TypedDict):
    question: str
    context: list[Document]
    answer: str

In [36]:
def generate(question: str, return_response: bool = False, **kwargs) -> State | (tuple[State, AIMessage]):
    retrieved_docs = vector_store.similarity_search(question)
    docs_content = "<context>\n\n"
    docs_content += "\n\n".join(f"<context{i+1}>{doc.page_content}</context{i+1}>" 
                               for i, doc in enumerate(retrieved_docs))
    docs_content += "\n\n</context>"

    message_prompt = {"question": question, "context": docs_content}
    messages = prompt.invoke(message_prompt)
    response = llm.invoke(messages)

    output = {
        "question": question,
        "context": retrieved_docs,
        "answer": response.content
    }
    
    if not return_response:
        return output
    
    return output, response

In [47]:
# Cambiando el tipo de prompt
def generate_without_rlm(question: str, return_response: bool = False, **kwargs) -> State | (tuple[State, AIMessage]):
    system_prompt: str = kwargs["system"] if "system" in kwargs else "Eres un asistente para responder preguntas" 
    historial: list = kwargs["historial"] if "historial" in kwargs else [(SystemMessage(system_prompt))]

    retrieved_docs = vector_store.similarity_search(question)
    docs_content = "<context>\n\n"
    docs_content += "\n\n".join(f"<context{i+1}>{doc.page_content}</context{i+1}>" 
                               for i, doc in enumerate(retrieved_docs))
    docs_content += "\n\n</context>"

    message_prompt = f"<question>{question}</question>\n\n{docs_content}"
    historial.append(HumanMessage(message_prompt))
    
    response = llm.invoke(historial)
    historial.append(response)

    output = {
        "question": question,
        "context": retrieved_docs,
        "answer": response.content
    }
    
    if not return_response:
        return output
    
    return output, response

## Pruebas

Aún queda hacer la creación más estrictas de pruebas, y hacer cambios

In [48]:
def show_answers(question: str, generate: callable, **kwargs) -> None:
    response = generate(question, False, **kwargs)

    print("### Question ###")
    print(response["question"])

    print("\n\n### Context ###")
    for i, doc in enumerate(response["context"]):
        print(f"<context{i+1}>")
        print(doc)
        print(f"</context{i+1}>\n")

    print("\n### Answer ###")
    print(response["answer"])

In [49]:
q1 = "¿Cuáles son los objetivos principales del proyecto Nebula?"
q2 = "Qué tecnologías se tuvieron en cuenta para Nebula?"
q3 = "Para qué sirvió el SAD en Nebula?"

### Con rlm

In [33]:
show_answers(q1, generate)

### Question ###
¿Cuáles son los objetivos principales del proyecto Nebula?


### Context ###
<context1>
page_content='Nebula est� dise�ado para ser escalable y modular, permitiendo una f�cil integraci�n de nuevas funcionalidades y mejoras a medida que evolucionen las necesidades de los usuarios.

#### Goals and Context

##### Objetivos del Proyecto

El objetivo principal de Nebula es proporcionar un sistema SaaS accesible y eficiente para la creaci�n y evaluaci�n de modelos de aprendizaje autom�tico, espec�ficamente para tareas de regresi�n y clasificaci�n. La plataforma est� dise�ada para usuarios con diversos niveles de experiencia en ciencia de datos, brindando una experiencia intuitiva y automatizada que permite crear modelos precisos sin necesidad de conocimientos avanzados en machine learning.

##### Objetivos Espec�ficos' metadata={'source': './info.md'}
</context1>

<context2>
page_content='### Solution Background

#### Architectural Approaches

Para garantizar que Nebula cump

In [34]:
show_answers(q2, generate)

### Question ###
Qué tecnologías se tuvieron en cuenta para Nebula?


### Context ###
<context1>
page_content='* Redis. Sistema de almacenamiento en cach� en memoria que se usa para mejorar el rendimiento del backend, especialmente en la gesti�n de sesiones de usuarios en Nebula.
* Scikit-learn. Biblioteca de Python para la creaci�n y entrenamiento de modelos de machine learning. En Nebula, se utiliza para entrenar y evaluar modelos de regresi�n y clasificaci�n.
* Zustand. Librer�a de gesti�n de estado para aplicaciones React que permite gestionar de manera eficiente el estado global de la aplicaci�n en Nebula sin necesidad de pasar datos a trav�s de propiedades entre componentes.
* Docker. Plataforma para el desarrollo, env�o y ejecuci�n de aplicaciones en contenedores. Se utiliza en Nebula para garantizar que los componentes del sistema funcionen de manera consistente en diferentes entornos.' metadata={'source': './info.md'}
</context1>

<context2>
page_content='### Solution Backgrou

In [35]:
show_answers(q3, generate)

### Question ###
Para qué sirvió el SAD en Nebula?


### Context ###
<context1>
page_content='# NEBULA

## Roadmap

### Purpose and Scope of the SAD

El SAD tiene como objetivo documentar la arquitectura de Nebula, detallando los componentes estructurales y sus interacciones para cumplir con los requisitos de desarrollo, escalabilidad y seguridad. En este documento se incluye toda la informaci�n sobre la arquitectura de software de Nebula, destacando aquellos aspectos que influyen en el rendimiento y la adaptabilidad del sistema. Las decisiones de dise�o documentadas en el SAD son aquellas que tienen un impacto arquitect�nico relevante en la estructura general del sistema, mientras que los detalles de implementaci�n espec�ficos se documentan en otros informes t�cnicos.

### How the SAD Is Organized' metadata={'source': './info.md'}
</context1>

<context2>
page_content='El SAD de Nebula est� estructurado en las siguientes secciones:  
Sección 1: Documentation Roadmap - Proporciona una v

### Custom

In [50]:
system_prompt = (
    "Eres un asistente encargado para resolver dudas del proyecto 'Nebula'" +
    ". En cada prompt que recibas, habrá un apartado donde estará la etiqueta <context>...</context>" +
    " donde tendrás el contexto de donde sacarás la información para responder la pregunta." +
    " Las respuestas deben ser lo más concisa y cortas posibles, y en caso de no saber algo" +
    " simplemente di que no lo sabes.\n" +
    "La pregunta la recibirás en las etiquetas <question>...</question>"
)

print(system_prompt)

historial = [SystemMessage(system_prompt)]

kwargs = {
    "system": system_prompt,
    "historial": historial
}

Eres un asistente encargado para resolver dudas del proyecto 'Nebula'. En cada prompt que recibas, habrá un apartado donde estará la etiqueta <context>...</context> donde tendrás el contexto de donde sacarás la información para responder la pregunta. Las respuestas deben ser lo más concisa y cortas posibles, y en caso de no saber algo simplemente di que no lo sabes.
La pregunta la recibirás en las etiquetas <question>...</question>


In [51]:
show_answers(q1, generate_without_rlm, **kwargs)

### Question ###
¿Cuáles son los objetivos principales del proyecto Nebula?


### Context ###
<context1>
page_content='Nebula est� dise�ado para ser escalable y modular, permitiendo una f�cil integraci�n de nuevas funcionalidades y mejoras a medida que evolucionen las necesidades de los usuarios.

#### Goals and Context

##### Objetivos del Proyecto

El objetivo principal de Nebula es proporcionar un sistema SaaS accesible y eficiente para la creaci�n y evaluaci�n de modelos de aprendizaje autom�tico, espec�ficamente para tareas de regresi�n y clasificaci�n. La plataforma est� dise�ada para usuarios con diversos niveles de experiencia en ciencia de datos, brindando una experiencia intuitiva y automatizada que permite crear modelos precisos sin necesidad de conocimientos avanzados en machine learning.

##### Objetivos Espec�ficos' metadata={'source': './info.md'}
</context1>

<context2>
page_content='### Solution Background

#### Architectural Approaches

Para garantizar que Nebula cump

In [52]:
show_answers(q2, generate_without_rlm, **kwargs)

### Question ###
Qué tecnologías se tuvieron en cuenta para Nebula?


### Context ###
<context1>
page_content='* Redis. Sistema de almacenamiento en cach� en memoria que se usa para mejorar el rendimiento del backend, especialmente en la gesti�n de sesiones de usuarios en Nebula.
* Scikit-learn. Biblioteca de Python para la creaci�n y entrenamiento de modelos de machine learning. En Nebula, se utiliza para entrenar y evaluar modelos de regresi�n y clasificaci�n.
* Zustand. Librer�a de gesti�n de estado para aplicaciones React que permite gestionar de manera eficiente el estado global de la aplicaci�n en Nebula sin necesidad de pasar datos a trav�s de propiedades entre componentes.
* Docker. Plataforma para el desarrollo, env�o y ejecuci�n de aplicaciones en contenedores. Se utiliza en Nebula para garantizar que los componentes del sistema funcionen de manera consistente en diferentes entornos.' metadata={'source': './info.md'}
</context1>

<context2>
page_content='### Solution Backgrou

In [53]:
show_answers(q3, generate_without_rlm, **kwargs)

### Question ###
Para qué sirvió el SAD en Nebula?


### Context ###
<context1>
page_content='# NEBULA

## Roadmap

### Purpose and Scope of the SAD

El SAD tiene como objetivo documentar la arquitectura de Nebula, detallando los componentes estructurales y sus interacciones para cumplir con los requisitos de desarrollo, escalabilidad y seguridad. En este documento se incluye toda la informaci�n sobre la arquitectura de software de Nebula, destacando aquellos aspectos que influyen en el rendimiento y la adaptabilidad del sistema. Las decisiones de dise�o documentadas en el SAD son aquellas que tienen un impacto arquitect�nico relevante en la estructura general del sistema, mientras que los detalles de implementaci�n espec�ficos se documentan en otros informes t�cnicos.

### How the SAD Is Organized' metadata={'source': './info.md'}
</context1>

<context2>
page_content='El SAD de Nebula est� estructurado en las siguientes secciones:  
Sección 1: Documentation Roadmap - Proporciona una v

In [54]:
display(kwargs["historial"])

[SystemMessage(content="Eres un asistente encargado para resolver dudas del proyecto 'Nebula'. En cada prompt que recibas, habrá un apartado donde estará la etiqueta <context>...</context> donde tendrás el contexto de donde sacarás la información para responder la pregunta. Las respuestas deben ser lo más concisa y cortas posibles, y en caso de no saber algo simplemente di que no lo sabes.\nLa pregunta la recibirás en las etiquetas <question>...</question>", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='<question>¿Cuáles son los objetivos principales del proyecto Nebula?</question>\n\n<context>\n\n<context1>Nebula est� dise�ado para ser escalable y modular, permitiendo una f�cil integraci�n de nuevas funcionalidades y mejoras a medida que evolucionen las necesidades de los usuarios.\n\n#### Goals and Context\n\n##### Objetivos del Proyecto\n\nEl objetivo principal de Nebula es proporcionar un sistema SaaS accesible y eficiente para la creaci�n y evaluaci�n de mode